In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition
warnings.filterwarnings('ignore')

# ===== データの読み込み =====
# train.csv: モデルを学習するためのデータ（正解ラベル付き）
# test.csv: 予測対象のデータ（正解ラベルなし）
train_df = pd.read_csv('/Users/masatsune/dsprog/train.csv')
test_df = pd.read_csv('/Users/masatsune/dsprog/test.csv')

print("📊 データの基本情報")
print(f"訓練データのサイズ: {train_df.shape[0]}行 × {train_df.shape[1]}列")
print(f"テストデータのサイズ: {test_df.shape[0]}行 × {test_df.shape[1]}列")

# 最初の数行を表示して、データの中身を確認
print("\n【訓練データの先頭5行】")
print(train_df.head())

# 予測したいターゲット（離職したかどうか）の分布を確認
print("\n【離職の分布】")
print(train_df['Attrition'].value_counts())
print(f"\n離職率: {(train_df['Attrition'] == 'Yes').sum() / len(train_df) * 100:.1f}%")

# どんな種類のデータがあるか確認
print("\n【データの型】")
print(train_df.dtypes)

# ===== IDと予測対象を分離 =====
# IDは予測に使わないので分けておく
train_ids = train_df['ID']
test_ids = test_df['ID']

# 予測対象（Attrition）を取り出す
# 'Yes' → 1（離職あり）、'No' → 0（離職なし）に変換
y = train_df['Attrition'].map({'Yes': 1, 'No': 0})

# 特徴量（予測に使う変数）を取り出す
# IDとAttritionは特徴量から除外
X = train_df.drop(['ID', 'Attrition'], axis=1)
X_test = test_df.drop(['ID'], axis=1)

print("データの前処理")
print(f"特徴量の数: {X.shape[1]}個")
print(f"データ数: 訓練{X.shape[0]}件、テスト{X_test.shape[0]}件")

# ===== カテゴリカル変数（文字列）を数値に変換 =====
# 例: 'Male' → 0、'Female' → 1 のように変換
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
print(f"\n文字列型の変数（{len(categorical_features)}個）:")
print(categorical_features)

# LabelEncoderで文字列を数値に変換
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    # 訓練データとテストデータを結合してから変換
    # こうすることで、テストデータに未知の値があっても対処できる
    combined = pd.concat([X[col], X_test[col]], axis=0)
    le.fit(combined)
    
    X[col] = le.transform(X[col])
    X_test[col] = le.transform(X_test[col])
    label_encoders[col] = le

print("\n✓ 文字列を数値に変換しました")

# ===== 役に立たない列を削除 =====
# EmployeeCountとStandardHoursは全員同じ値なので削除
# Over18も全員'Y'なので削除
useless_cols = ['EmployeeCount', 'StandardHours', 'Over18']
useless_cols = [col for col in useless_cols if col in X.columns]
X = X.drop(useless_cols, axis=1)
X_test = X_test.drop(useless_cols, axis=1)

print(f"\n不要な列を削除しました: {useless_cols}")
print(f"最終的な特徴量の数: {X.shape[1]}個")

# ===== 訓練データを分割 =====
# 訓練データの一部を検証用に取っておく（モデルの性能を確認するため）
# 80%を訓練、20%を検証に使用
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"訓練データ: {X_train.shape[0]}件")
print(f"検証データ: {X_valid.shape[0]}件")

#主成分分析
pca = decomposition.PCA(n_components=2)
X_pca = pca.fit_transform(X)
print(pca.components_)

#パラメータチューニング
param = {
    'max_depth':[5, 10, 15, 20, 30, None],
    'n_estimators':[100, 200, 300, 500, 800],
    'max_features':["sqrt", "log2", 0.3, 0.5, 0.7, 1.0],
    'min_samples_leaf':[1, 2, 4, 8]}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param, cv=5)
grid_search.fit(X_train, y_train)

# ===== ランダムフォレストモデルを作成 =====
# ランダムフォレスト: 
# max_iter: 学習の反復回数
# random_state: 結果を再現可能にするための乱数シード
model = RandomForestClassifier(n_estimators=1000)


# モデルを訓練（学習）
print("\n学習中...")
model.fit(X_train, y_train)
print("✓ 学習完了！")

# ===== モデルの性能を評価 =====
# 検証データで予測を行う
y_pred = model.predict(X_valid)

# 正解率を計算
accuracy = accuracy_score(y_valid, y_pred)
print("モデルの性能")
print(f"正解率: {accuracy:.4f} ({accuracy*100:.2f}%)")

# 混同行列: どんな間違いをしているか確認
cm = confusion_matrix(y_valid, y_pred)
print("\n【混同行列】")
print(f"正しく「離職なし」と予測: {cm[0,0]}件")
print(f"誤って「離職あり」と予測: {cm[0,1]}件")
print(f"誤って「離職なし」と予測: {cm[1,0]}件")
print(f"正しく「離職あり」と予測: {cm[1,1]}件")

# 詳細なレポート
print("\n【詳細レポート】")
print(classification_report(y_valid, y_pred, target_names=['No', 'Yes']))

# ===== テストデータで予測 =====
# テストデータ全体を使って予測を行う
# predict()は0か1を返す
test_predictions = model.predict(X_test)

# 予測結果を確認
print(f"\n予測完了: {len(test_predictions)}件")
print(f"離職ありと予測: {(test_predictions == 1).sum()}件")
print(f"離職なしと予測: {(test_predictions == 0).sum()}件")

📊 データの基本情報
訓練データのサイズ: 882行 × 36列
テストデータのサイズ: 588行 × 35列

【訓練データの先頭5行】
   ID  Age Attrition     BusinessTravel  DailyRate              Department  \
0   1   41       Yes      Travel_Rarely       1102                   Sales   
1   2   49        No  Travel_Frequently        279  Research & Development   
2   5   27        No      Travel_Rarely        591  Research & Development   
3   6   32        No  Travel_Frequently       1005  Research & Development   
4   8   30        No      Travel_Rarely       1358  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  ...  \
0                 1          2  Life Sciences              1  ...   
1                 8          1  Life Sciences              1  ...   
2                 2          1        Medical              1  ...   
3                 2          2  Life Sciences              1  ...   
4                24          1  Life Sciences              1  ...   

   RelationshipSatisfaction  StandardHours Sto